In [260]:
import polars as pl
from glob import glob
import sys
import json

In [267]:
# cohort_id	condition_name	meddra_pt_id	meddra_pt_term	Positive Controls (N)
meddra_mappings = (pl.read_csv('../data/onsides_mapping.csv')
                   .select(
                       pl.col('cohort_id'),
                       pl.col('condition_name').str.to_lowercase(),
                       pl.col('meddra_pt_id').alias('pt_meddra_id'),
                       pl.col('meddra_pt_term').str.to_lowercase().alias('pt_meddra_term'),
                       pl.col('Positive Controls (N)').alias('positive_controls')               
                       )
                   )
meddra_mappings.head()

cohort_id,condition_name,pt_meddra_id,pt_meddra_term,positive_controls
i64,str,i64,str,i64
500000401,"""acute kidney i…",10038435,"""renal failure""",20
500000401,"""acute kidney i…",10062237,"""renal impairme…",8
500000401,"""acute kidney i…",10069339,"""acute kidney i…",6
500000301,"""acute liver in…",10000804,"""acute hepatic …",5
500000301,"""acute liver in…",10019663,"""hepatic failur…",45


## Get Labels with ADRs from Reference Set

Reference set contains: AKI, ALI, AMI, and GIB

In [321]:
adr_section = (
    pl.read_csv('/Users/undinagisladottir/Documents/Columbia/Tatonetti_Lab/20231113_onsides/adverse_reactions.csv',
                          dtypes={
                            'pt_meddra_id': pl.Int64,
                            'pt_meddra_term': pl.Utf8,
                            'num_ingredients':  pl.Int64,
                            'ingredients_rxcuis': pl.Utf8,
                            'ingredients_names': pl.Utf8
                          })
                          .filter(pl.col('num_ingredients') == 1)
                          .select(
                              pl.col('pt_meddra_id'),
                              pl.col('pt_meddra_term').str.to_lowercase(),
                              pl.col('ingredients_rxcuis').cast(pl.Int64),
                              pl.col('ingredients_names').str.to_lowercase()
                              )
)
adr_section.head()


pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names
i64,str,i64,str
10000059,"""abdominal disc…",6916,"""metolazone"""
10000060,"""abdominal dist…",6916,"""metolazone"""
10000081,"""abdominal pain…",6916,"""metolazone"""
10001507,"""agranulocytosi…",6916,"""metolazone"""
10001682,"""alkalosis hypo…",6916,"""metolazone"""


In [348]:
merged_data = (
    adr_section
    .join(meddra_mappings, on=["pt_meddra_id", "pt_meddra_term"], how='inner')
    )
merged_data.head()
merged_data.filter(pl.col('ingredients_names') == 'candesartan')

pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names,cohort_id,condition_name,positive_controls
i64,str,i64,str,i64,str,i64
10028596,"""myocardial inf…",214354,"""candesartan""",500000801,"""acute myocardi…",29
10038435,"""renal failure""",214354,"""candesartan""",500000401,"""acute kidney i…",20
10062237,"""renal impairme…",214354,"""candesartan""",500000401,"""acute kidney i…",8


In [269]:
rxnorm_2024_mappings = pl.read_csv('../data/20240312/rxnorm_mappings.csv').select(
    pl.col('SETID').alias('set_id'),
    pl.col('SPL_VERSION').alias('spl_version'),
    pl.col('RXCUI'), #.alias('ingredients_rxcuis'),
    pl.col('RXSTRING'),   
)
rxnorm_ingredient = pl.read_csv('../data/20240312/rxnorm_product_to_ingredient.csv')
rxnorm_2024_mappings.head()
rxnorm_ingredient.head()

product_rx_cui,product_name,product_omop_concept_id,ingredient_rx_cui,ingredient_name,ingredient_omop_concept_id
i64,str,i64,i64,str,i64
1000000,"""amlodipine 5 M…",40224176,17767,"""amlodipine""",1332418
1000000,"""amlodipine 5 M…",40224176,5487,"""hydrochlorothi…",974166
1000000,"""amlodipine 5 M…",40224176,321064,"""olmesartan""",40226742
1000001,"""amlodipine 5 M…",40224178,17767,"""amlodipine""",1332418
1000001,"""amlodipine 5 M…",40224178,5487,"""hydrochlorothi…",974166


In [349]:
product_2024 = (rxnorm_2024_mappings.join(rxnorm_ingredient,
                           right_on=['product_rx_cui'],
                           left_on = 'RXCUI', how='inner')
                     .select(
                         pl.col('set_id').alias('SETID'),
                         pl.col('spl_version').alias('SPL_VERSION'),
                         pl.col('ingredient_rx_cui'),
                         pl.col('ingredient_name')
                     )
)
product_2024.head()

set_id,spl_version,RXCUI,RXSTRING,product_name,product_omop_concept_id,ingredient_rx_cui,ingredient_name,ingredient_omop_concept_id
str,i64,i64,str,str,i64,i64,str,i64
"""000155a8-709c-…",27,198014,"""naproxen 500 M…","""naproxen 500 M…",19019273,7258,"""naproxen""",1115008
"""000155a8-709c-…",27,198014,"""naproxen 500 M…","""naproxen 500 M…",19019273,7258,"""naproxen""",1115008
"""0001d1cf-f727-…",2,1046593,"""benzalkonium c…","""benzalkonium c…",40229527,1378,"""benzalkonium""",916460
"""0001d1cf-f727-…",2,1046593,"""benzalkonium c…","""benzalkonium c…",40229527,1378,"""benzalkonium""",916460
"""0001d1cf-f727-…",2,1046593,"""benzalkonium c…","""benzalkonium c…",40229527,1378,"""benzalkonium""",916460


In [329]:
product_2024_setids = (adr_section.join(meddra_mappings,
                 on=["pt_meddra_id", "pt_meddra_term"],
                 how='inner')
            .join(product_2024, # set_id	spl_version	ingredient_rx_cui	ingredient_name
                  right_on=['ingredient_rx_cui', 'ingredient_name'],
                  left_on=['ingredients_rxcuis', 'ingredients_names']
))
# .join(rxnorm_2024_mappings,
#       on=['set_id', 'spl_version'],
#       how='inner').select(
#     pl.col('set_id'),
#     pl.col('spl_version'),
#     pl.col('ingredients_names'),
#     pl.col('ingredients_rxcuis'),
#     pl.col('RXSTRING'),
#     pl.col('RXCUI')
# )).head()
product_2024_setids.head()

pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names,cohort_id,condition_name,positive_controls,SETID,SPL_VERSION
i64,str,i64,str,i64,str,i64,str,i64
10017955,"""gastrointestin…",7258,"""naproxen""",500001001,"""gi bleed""",21,"""000155a8-709c-…",27
10018830,"""haematemesis""",7258,"""naproxen""",500001001,"""gi bleed""",11,"""000155a8-709c-…",27
10019663,"""hepatic failur…",7258,"""naproxen""",500000301,"""acute liver in…",45,"""000155a8-709c-…",27
10024690,"""liver function…",7258,"""naproxen""",500000301,"""acute liver in…",15,"""000155a8-709c-…",27
10028596,"""myocardial inf…",7258,"""naproxen""",500000801,"""acute myocardi…",29,"""000155a8-709c-…",27


In [352]:
# map set_id and spl_version to file name
label_dir = '../data/2024_latest_labels/'
active_labels = glob(label_dir+'*/*.json')
all_labels = []
count = 0
for label in active_labels:
    # Opening JSON file
    f = open(label)
    data = json.load(f)
    count += 1
    all_labels.append([label, data['set_id'], data['spl_version'], data['title']])

In [353]:
label_df = pl.DataFrame(all_labels, schema=[("file", pl.Utf8),
                                            ("set_id", pl.Utf8),
                                            ("spl_version", pl.Int64),
                                            ("title", pl.Utf8)])
label_df.head()

file,set_id,spl_version,title
str,str,i64,str
"""../data/2024_l…","""df12ceb2-5b4b-…",19,"""These highligh…"
"""../data/2024_l…","""0ab861c2-d5ca-…",1,"""These highligh…"
"""../data/2024_l…","""a88ac1b4-e2c9-…",41,"""These highligh…"
"""../data/2024_l…","""f7b3f443-e83d-…",49,"""These highligh…"
"""../data/2024_l…","""a398400e-bd31-…",14,"""These highligh…"


In [405]:
(label_df.join(product_2024, on=["set_id", "spl_version"],
               how='inner')
               .filter(
                       (pl.col('title').str.contains('candesartan'))
                   )
)

file,set_id,spl_version,title,RXCUI,RXSTRING,product_name,product_omop_concept_id,ingredient_rx_cui,ingredient_name,ingredient_omop_concept_id
str,str,i64,str,i64,str,str,i64,i64,str,i64
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",153822,"""candesartan ci…","""candesartan ci…",1351558,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",153822,"""candesartan ci…","""candesartan ci…",1351558,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",153823,"""candesartan ci…","""candesartan ci…",1351559,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",153823,"""candesartan ci…","""candesartan ci…",1351559,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",577776,"""candesartan ci…","""candesartan ci…",1351583,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",577776,"""candesartan ci…","""candesartan ci…",1351583,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",639537,"""candesartan ci…","""candesartan ci…",1351587,214354,"""candesartan""",1351557
"""../data/2024_l…","""1c90aa68-cc12-…",11,"""These highligh…",639537,"""candesartan ci…","""candesartan ci…",1351587,214354,"""candesartan""",1351557


In [227]:
# 280213 --- 70137
(product_2024_setids.join(label_df,
                        right_on=["set_id", "spl_version"],
                        left_on=["SETID", "SPL_VERSION"],
                        how='inner')
        .select(
            pl.col('SETID'),
            pl.col('SPL_VERSION'),
            pl.col('ingredients_names'),
            pl.col('ingredients_rxcuis'),
            pl.col('condition_name'),
            pl.col('pt_meddra_id'),
            pl.col('pt_meddra_term'),
            pl.col('file'),
            pl.col('title')
        ).unique()
).write_csv('../data/labels_for_rag.csv')

In [361]:
product_mappings = (product_2024_setids.join(label_df,
                        right_on=["set_id", "spl_version"],
                        left_on=["SETID", "SPL_VERSION"],
                        how='inner')
)
product_mappings.head()

pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names,cohort_id,condition_name,positive_controls,SETID,SPL_VERSION,file,title
i64,str,i64,str,i64,str,i64,str,i64,str,str
10018830,"""haematemesis""",1425099,"""trametinib""",500001001,"""gi bleed""",11,"""0002ad27-779d-…",22,"""../data/2024_l…","""These highligh…"
10018830,"""haematemesis""",1425099,"""trametinib""",500001001,"""gi bleed""",11,"""0002ad27-779d-…",22,"""../data/2024_l…","""These highligh…"
10038435,"""renal failure""",1425099,"""trametinib""",500000401,"""acute kidney i…",20,"""0002ad27-779d-…",22,"""../data/2024_l…","""These highligh…"
10038435,"""renal failure""",1425099,"""trametinib""",500000401,"""acute kidney i…",20,"""0002ad27-779d-…",22,"""../data/2024_l…","""These highligh…"
10018830,"""haematemesis""",1425099,"""trametinib""",500001001,"""gi bleed""",11,"""0002ad27-779d-…",22,"""../data/2024_l…","""These highligh…"


In [362]:
product_mappings.filter(pl.col('ingredients_names') == 'candesartan').head()

pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names,cohort_id,condition_name,positive_controls,SETID,SPL_VERSION,file,title
i64,str,i64,str,i64,str,i64,str,i64,str,str
10028596,"""myocardial inf…",214354,"""candesartan""",500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11,"""../data/2024_l…","""These highligh…"
10038435,"""renal failure""",214354,"""candesartan""",500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11,"""../data/2024_l…","""These highligh…"
10062237,"""renal impairme…",214354,"""candesartan""",500000401,"""acute kidney i…",8,"""1c90aa68-cc12-…",11,"""../data/2024_l…","""These highligh…"
10028596,"""myocardial inf…",214354,"""candesartan""",500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11,"""../data/2024_l…","""These highligh…"
10038435,"""renal failure""",214354,"""candesartan""",500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11,"""../data/2024_l…","""These highligh…"


In [360]:
pryan_set = (pl.read_csv('../data/pryan_reference_set_ades.csv')
             .select(
                 pl.col('cohort_id'), 
                 pl.col('condition_name').str.to_lowercase(),
                 'drug_concept_id',
                 pl.col('drug_name').str.to_lowercase(),
                 'affect'
             )
             )
pryan_set.head()

cohort_id,condition_name,drug_concept_id,drug_name,affect
i64,str,i64,str,i64
500000401,"""acute kidney i…",1703687,"""acyclovir""",1
500000401,"""acute kidney i…",1167322,"""allopurinol""",1
500000401,"""acute kidney i…",19026710,"""capreomycin""",1
500000401,"""acute kidney i…",1340128,"""captopril""",1
500000401,"""acute kidney i…",992590,"""chlorothiazide…",1


In [381]:
(
    pryan_set
    .with_columns(
        reference = 1
    )
    .join(
        product_mappings.with_columns(product = 1),
        left_on = ['drug_name', 'condition_name'],
        right_on = ['ingredients_names', 'condition_name'],
        how='left'
    )
    .select(
        'drug_name',
        'condition_name',
        'affect',
        'reference',
        'product'
    )
    .unique(['drug_name', 'condition_name', 'affect', 'product'])
    .fill_null(0)
    .groupby(['condition_name', 'affect']).agg(pl.col('reference').sum(),
                                     pl.col('product').sum())
    .with_columns(
        (pl.col('reference') - pl.col('product')).alias('missed')
    )
)

/var/folders/sv/s1jkxt011xn9v02w3w30vx640000gn/T/ipykernel_77975/1849558158.py:21: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(['condition_name', 'affect']).agg(pl.col('reference').sum(),


condition_name,affect,reference,product,missed
str,i64,i32,i32,i32
"""acute liver in…",0,37,2,35
"""gi bleed""",0,67,2,65
"""acute kidney i…",1,24,22,2
"""acute liver in…",1,81,59,22
"""acute kidney i…",0,64,2,62
"""acute myocardi…",1,36,28,8
"""acute myocardi…",0,66,0,66
"""gi bleed""",1,24,21,3


In [367]:
(
    pryan_set
    .with_columns(
        reference = 1
    )
    .join(
        product_mappings.with_columns(product = 1),
        left_on = ['drug_name', 'condition_name'],
        right_on = ['ingredients_names', 'condition_name'],
        how='left'
    )
    .select(
        'condition_name',
        'drug_name',
        'affect',
        'reference',
        'product'
    )
    .unique(['drug_name', 'condition_name', 'affect', 'product'])
    .fill_null(0)
    .filter(pl.col('affect') == 1)
).sort(['condition_name', 'drug_name'])

condition_name,drug_name,affect,reference,product
str,str,i64,i32,i32
"""acute kidney i…","""acyclovir""",1,1,1
"""acute kidney i…","""allopurinol""",1,1,1
"""acute kidney i…","""candesartan""",1,1,1
"""acute kidney i…","""capreomycin""",1,1,0
"""acute kidney i…","""captopril""",1,1,1
"""acute kidney i…","""chlorothiazide…",1,1,1
"""acute kidney i…","""cyclosporine""",1,1,1
"""acute kidney i…","""diflunisal""",1,1,1
"""acute kidney i…","""enalaprilat""",1,1,1


In [368]:
(
    pryan_set
    .filter(
        pl.col('affect') == 1
    ).select('drug_name').unique()
    .join(
        product_mappings.with_columns(label = 1),
        left_on = ['drug_name'],
        right_on = ['ingredients_names'],
        how='inner'
    ).select('file').unique()
)

file
str
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"
"""../data/2024_l…"


In [370]:
pos_cont_drugs = pryan_set.filter(pl.col('affect') == 1).select('drug_name').unique()
pos_cont_drugs.shape

(123, 1)

In [379]:
num_titles = set()
for label in label_df.iter_rows():
    is_in_title = [x.lower() in label[-1].lower() for x in pos_cont_drugs['drug_name'].to_list()]
    if any(is_in_title):
        num_titles.add(label[-1])

len(num_titles)

114

In [315]:
for label in label_df.iter_rows():
    if 'candesartan' in label[-1].lower():
        print(label)

('../data/2024_latest_labels/latest_labels_ingredients/20240221_1c90aa68-cc12-41ef-aebc-2616ae45683a.json', '1c90aa68-cc12-41ef-aebc-2616ae45683a', 11, 'These highlights do not include all the information needed to use \n\ncandesartan cilexetil \n\n safely and effectively.  See full prescribing information for \n\ncandesartan cilexetil \n\n.\n\nCandesartan cilexetil tablets, for oral use\nInitial U.S. Approval: 1998')


In [325]:
product_2024_setids.filter(pl.col('SETID') == '1c90aa68-cc12-41ef-aebc-2616ae45683a').head()

pt_meddra_id,pt_meddra_term,ingredients_rxcuis,ingredients_names,cohort_id,condition_name,positive_controls,SETID,SPL_VERSION
i64,str,i64,str,i64,str,i64,str,i64
10028596,"""myocardial inf…",214354,"""candesartan""",500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11
10038435,"""renal failure""",214354,"""candesartan""",500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11
10062237,"""renal impairme…",214354,"""candesartan""",500000401,"""acute kidney i…",8,"""1c90aa68-cc12-…",11
10028596,"""myocardial inf…",214354,"""candesartan""",500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11
10038435,"""renal failure""",214354,"""candesartan""",500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11


In [378]:
pryan_set.filter(pl.col('drug_name') == 'candesartan').join(
    product_2024_setids.filter(pl.col('SETID') == '1c90aa68-cc12-41ef-aebc-2616ae45683a').head(),
    left_on = 'drug_name',
    right_on='ingredients_names'
)

cohort_id,condition_name,drug_concept_id,drug_name,affect,pt_meddra_id,pt_meddra_term,ingredients_rxcuis,cohort_id_right,condition_name_right,positive_controls,SETID,SPL_VERSION
i64,str,i64,str,i64,i64,str,i64,i64,str,i64,str,i64
500000401,"""acute kidney i…",1351557,"""candesartan""",1,10028596,"""myocardial inf…",214354,500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11
500000401,"""acute kidney i…",1351557,"""candesartan""",1,10038435,"""renal failure""",214354,500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11
500000401,"""acute kidney i…",1351557,"""candesartan""",1,10062237,"""renal impairme…",214354,500000401,"""acute kidney i…",8,"""1c90aa68-cc12-…",11
500000401,"""acute kidney i…",1351557,"""candesartan""",1,10028596,"""myocardial inf…",214354,500000801,"""acute myocardi…",29,"""1c90aa68-cc12-…",11
500000401,"""acute kidney i…",1351557,"""candesartan""",1,10038435,"""renal failure""",214354,500000401,"""acute kidney i…",20,"""1c90aa68-cc12-…",11
